In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
data_sliced = pd.read_csv('./data/dataset_sliced.csv')


In [4]:
X = data_sliced.iloc[:, 0:17]
y = data_sliced.iloc[:, 18]


In [5]:
y.astype(np.float32)
X.astype(np.float32)

,chest_ACC_x,chest_ACC_y,chest_ACC_z,Resp,ECG,wrist_ACC_x,wrist_ACC_y,wrist_ACC_z,wrist_BVP,wrist_TEMP,WEIGHT,Gender,AGE,HEIGHT,SKIN,SPORT,Rpeaks
0,0.8564,-0.0678,-0.3656,4.441833,0.015610,-0.765625,-0.078125,0.671875,7.280000,32.130001,78.0,1.0,34.0,182.0,3.0,6.0,0.0
1,0.8520,-0.0666,-0.3826,2.915955,0.159714,-0.750000,-0.078125,0.671875,6.340000,32.150002,78.0,1.0,34.0,182.0,3.0,6.0,1.0
2,0.8300,-0.0102,-0.3502,-1.623535,0.014603,-1.046875,-0.046875,0.843750,7.700000,32.150002,78.0,1.0,34.0,182.0,3.0,6.0,0.0
3,0.7630,0.0552,-0.6070,0.416565,-0.307617,-0.859375,-0.390625,0.390625,-120.349998,32.150002,78.0,1.0,34.0,182.0,3.0,6.0,0.0
4,0.8894,-0.0520,-0.2520,0.198364,0.604111,-1.015625,-0.078125,0.140625,-7.060000,32.150002,78.0,1.0,34.0,182.0,3.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64692,0.7874,-0.0146,-0.5400,1.446533,0.486740,-0.984375,0.156250,0.234375,7.300000,34.029999,79.0,1.0,28.0,183.0,2.0,5.0,1.0
64693,0.8936,-0.1080,0.4138,10.545349,-0.124466,-0.562500,-0.031250,0.718750,117.290001,34.000000,79.0,1.0,28.0,183.0,2.0,5.0,0.0
64694,0.9012,0.0262,-0.1576,-7.260132,-0.138702,-0.375000,-0.203125,0.875000,39.549999,34.000000,79.0,1.0,28.0,183.0,2.0,5.0,1.0
64695,0.8966,0.0104,-0.2252,1.187134,-0.288849,-1.421875,-0.046875,0.171875,-21.280001,34.000000,79.0,1.0,28.0,183.0,2.0,5.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train,  y_val = train_test_split(X, y, test_size=0.4, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(38818, 17) (38818,) (25879, 17) (25879,)


In [7]:
## Change shape
X_train = np.expand_dims(X_train, -1)
X_val = np.expand_dims(X_val, -1)

print(X_train.shape, X_val.shape)

input_dim = X_train.shape[1:]

(38818, 17, 1) (25879, 17, 1)


In [22]:
input_dim

(17, 1)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, Activation, Flatten, Dense, LeakyReLU, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop, Adam

In [9]:
model = Sequential()
model.add(Convolution1D(filters=64, kernel_size=2, input_shape=input_dim))
model.add(MaxPooling1D(pool_size=2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation="elu"))
model.compile(optimizer=Adam(learning_rate=0.01), loss="mean_absolute_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 16, 64)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 8, 64)            0         
 )                                                               
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 449
Trainable params: 449
Non-trainable pa

In [16]:
history = model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=1, validation_data=(X_val, y_val))

Epoch 1/100
389/389 [==============================] - 1s 2ms/step - loss: 14.5237 - val_loss: 14.2274
Epoch 2/100
389/389 [==============================] - 1s 2ms/step - loss: 14.5085 - val_loss: 14.2683
Epoch 3/100
389/389 [==============================] - 1s 2ms/step - loss: 14.4561 - val_loss: 15.4725
Epoch 4/100
389/389 [==============================] - 1s 2ms/step - loss: 14.5330 - val_loss: 14.2267
Epoch 5/100
389/389 [==============================] - 1s 2ms/step - loss: 14.5367 - val_loss: 15.0199
Epoch 6/100
389/389 [==============================] - 1s 2ms/step - loss: 14.5266 - val_loss: 14.2101
Epoch 7/100
389/389 [==============================] - 1s 2ms/step - loss: 14.4702 - val_loss: 14.1313
Epoch 8/100
389/389 [==============================] - 1s 2ms/step - loss: 14.4928 - val_loss: 14.1899
Epoch 9/100
389/389 [==============================] - 1s 2ms/step - loss: 14.3715 - val_loss: 14.0239
Epoch 10/100
389/389 [==============================] - 1s 2ms/step - los

In [17]:
model.save('./.ipynb_checkpoints/CNN.hdf5')

# model = load_model('./.ipynb_checkpoints/LSTM.hdf5')



In [18]:
model.evaluate(X_val, y_val)

809/809 [==============================] - 0s 409us/step - loss: 14.2401


14.240123748779297

In [19]:
S1 = data_sliced.loc[data_sliced['Subject'] == 'S1']
S2 = data_sliced.loc[data_sliced['Subject'] == 'S2']
S3 = data_sliced.loc[data_sliced['Subject'] == 'S3']
S4 = data_sliced.loc[data_sliced['Subject'] == 'S4']
S5 = data_sliced.loc[data_sliced['Subject'] == 'S5']
S6 = data_sliced.loc[data_sliced['Subject'] == 'S6']
S7 = data_sliced.loc[data_sliced['Subject'] == 'S7']
S8 = data_sliced.loc[data_sliced['Subject'] == 'S8']
S9 = data_sliced.loc[data_sliced['Subject'] == 'S9']
S10 = data_sliced.loc[data_sliced['Subject'] == 'S10']
S11 = data_sliced.loc[data_sliced['Subject'] == 'S11']
S12 = data_sliced.loc[data_sliced['Subject'] == 'S12']
S13 = data_sliced.loc[data_sliced['Subject'] == 'S13']
S14 = data_sliced.loc[data_sliced['Subject'] == 'S14']
S15 = data_sliced.loc[data_sliced['Subject'] == 'S15']

S1 = data_sliced.loc[data_sliced['Subject'] == 'S1']
S1_X = S1.iloc[:, 0:17]
S1_y = S1.iloc[:, 18]
S2 = data_sliced.loc[data_sliced['Subject'] == 'S2']
S2_X = S2.iloc[:, 0:17]
S2_y = S2.iloc[:, 18]
S3 = data_sliced.loc[data_sliced['Subject'] == 'S3']
S3_X = S3.iloc[:, 0:17]
S3_y = S3.iloc[:, 18]
S4 = data_sliced.loc[data_sliced['Subject'] == 'S4']
S4_X = S4.iloc[:, 0:17]
S4_y = S4.iloc[:, 18]
S5 = data_sliced.loc[data_sliced['Subject'] == 'S5']
S5_X = S5.iloc[:, 0:17]
S5_y = S5.iloc[:, 18]
S6 = data_sliced.loc[data_sliced['Subject'] == 'S6']
S6_X = S6.iloc[:, 0:17]
S6_y = S6.iloc[:, 18]
S7 = data_sliced.loc[data_sliced['Subject'] == 'S7']
S7_X = S7.iloc[:, 0:17]
S7_y = S7.iloc[:, 18]
S8 = data_sliced.loc[data_sliced['Subject'] == 'S8']
S8_X = S8.iloc[:, 0:17]
S8_y = S8.iloc[:, 18]
S9 = data_sliced.loc[data_sliced['Subject'] == 'S9']
S9_X = S9.iloc[:, 0:17]
S9_y = S9.iloc[:, 18]
S10 = data_sliced.loc[data_sliced['Subject'] == 'S10']
S10_X = S10.iloc[:, 0:17]
S10_y = S10.iloc[:, 18]
S11 = data_sliced.loc[data_sliced['Subject'] == 'S11']
S11_X = S11.iloc[:, 0:17]
S11_y = S11.iloc[:, 18]
S12 = data_sliced.loc[data_sliced['Subject'] == 'S12']
S12_X = S12.iloc[:, 0:17]
S12_y = S12.iloc[:, 18]
S13 = data_sliced.loc[data_sliced['Subject'] == 'S13']
S13_X = S13.iloc[:, 0:17]
S13_y = S13.iloc[:, 18]
S14 = data_sliced.loc[data_sliced['Subject'] == 'S14']
S14_X = S14.iloc[:, 0:17]
S14_y = S14.iloc[:, 18]
S15 = data_sliced.loc[data_sliced['Subject'] == 'S15']
S15_X = S15.iloc[:, 0:17]
S15_y = S15.iloc[:, 18]




In [20]:
model.evaluate(S1_X,S1_y)
model.evaluate(S2_X,S2_y)
model.evaluate(S3_X,S3_y)
model.evaluate(S4_X,S4_y)
model.evaluate(S5_X,S5_y)
model.evaluate(S6_X,S6_y)
model.evaluate(S7_X,S7_y)
model.evaluate(S8_X,S8_y)
model.evaluate(S9_X,S9_y)
model.evaluate(S10_X,S10_y)
model.evaluate(S11_X,S11_y)
model.evaluate(S12_X,S12_y)
model.evaluate(S13_X,S13_y)
model.evaluate(S14_X,S14_y)
model.evaluate(S15_X,S15_y)

124/124 [==============================] - 0s 591us/step - loss: 12.8615


12.861470222473145

In [21]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,4))
plt.grid(True)
plt.plot( model.predict(S1_X), label = 'prediected CNN')
plt.plot( S1_y, label = 'Ground truth')
plt.legend()
plt.ylabel('Heart rate - bpm', fontsize=12)
plt.xlabel('Time - s')
plt.show()